In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():

    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [2]:
df = pd.read_csv('data_processing/csv_file/landmark_6_9_3.csv')

# Define your PyTorch dataset class
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)

train_size = int(0.8 * len(keypoint_dataset))
val_size = len(keypoint_dataset) - train_size

train_dataset, val_dataset = random_split(keypoint_dataset, [train_size, val_size])

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 1999
Validation dataset size: 500


In [3]:
import model as m
model = m.SimpleNN3().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Model's state_dict:
fc1.weight 	 torch.Size([63, 126])
fc1.bias 	 torch.Size([63])
fc2.weight 	 torch.Size([40, 63])
fc2.bias 	 torch.Size([40])
fc3.weight 	 torch.Size([20, 40])
fc3.bias 	 torch.Size([20])
fc4.weight 	 torch.Size([5, 20])
fc4.bias 	 torch.Size([5])


In [4]:
num_epochs = 100
previous_loss = float('inf')  # Initialize with a high value
writer = SummaryWriter()
val_loss = 0.0
for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct = 0
    total_samples = 0
    if epoch != 0 and epoch % 10 == 0:
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_total_correct = 0
        val_total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['data']
                labels = batch['label']

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

                _, predicted = torch.max(outputs, 1)
                val_total_samples += labels.size(0)
                val_total_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_total_correct / val_total_samples
        print(f'Validation Loss: {val_loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')
        writer.add_scalar('Validation Loss', val_loss.item(), epoch)
        writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    for batch in train_loader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    writer.add_scalar('Training Loss', loss.item(), epoch)
    writer.add_scalar('Training Accuracy', train_accuracy, epoch)
    if val_loss < previous_loss:
        previous_loss = val_loss  # Update the previous loss

# Save the trained model
torch.save(model.state_dict(), "models/6_9_4.pth")

Epoch [1/100], Training Loss: 1.5986
Epoch [2/100], Training Loss: 1.5159
Epoch [3/100], Training Loss: 1.4402
Epoch [4/100], Training Loss: 1.2582
Epoch [5/100], Training Loss: 1.1318
Epoch [6/100], Training Loss: 1.0619
Epoch [7/100], Training Loss: 1.0233
Epoch [8/100], Training Loss: 1.0162
Epoch [9/100], Training Loss: 0.9353
Epoch [10/100], Training Loss: 0.9535
Validation Loss: 0.9134, Training Accuracy: 95.80%, Validation Accuracy: 99.40%
Epoch [11/100], Training Loss: 0.9062
Epoch [12/100], Training Loss: 0.9588
Epoch [13/100], Training Loss: 0.9493
Epoch [14/100], Training Loss: 0.9441
Epoch [15/100], Training Loss: 0.9305
Epoch [16/100], Training Loss: 0.9175
Epoch [17/100], Training Loss: 0.9383
Epoch [18/100], Training Loss: 0.9188
Epoch [19/100], Training Loss: 0.9265
Epoch [20/100], Training Loss: 0.9332
Validation Loss: 0.9110, Training Accuracy: 97.80%, Validation Accuracy: 99.40%
Epoch [21/100], Training Loss: 0.9177
Epoch [22/100], Training Loss: 0.9162
Epoch [23/100

Run this in terminal to open tensorboard: tensorboard --logdir=runs
